In [ ]:
import torch
from models.dcnn_lrelu import DCNN_LReLU

from monai.data import DataLoader, decollate_batch
from monai.utils import set_determinism
from monai.data.utils import list_data_collate
from monai.metrics import MSEMetric, ROCAUCMetric
from monai.transforms import (
    Activations,
    AsDiscrete,
)

import config
import misc
import load_data
import process_data
import data_preproc.data_preproc_config as data_preproc_config


: 

In [ ]:
wandb_mode = config.wandb_mode
torch_version = config.torch_version
seed = config.seed
device = config.device
cudnn_benchmark = config.cudnn_benchmark
exp_dir = config.exp_dir
filename_stratified_sampling_test_csv = config.filename_stratified_sampling_test_csv
filename_stratified_sampling_full_csv = config.filename_stratified_sampling_full_csv
perform_stratified_sampling_full = config.perform_stratified_sampling_full
sampling_type = config.sampling_type
cv_folds = config.cv_folds
cv_type = config.cv_type
use_sampler = config.use_sampler
perform_data_aug = config.perform_data_aug
rand_cropping_size = config.rand_cropping_size
input_size = config.input_size
resize_mode = config.resize_mode
seg_orig_labels = config.seg_orig_labels
seg_target_labels = config.seg_target_labels
# Deep Learning model
dataloader_drop_last = config.dataloader_drop_last
data_aug_p = config.data_aug_p
data_aug_strength = config.data_aug_strength
perform_augmix = config.perform_augmix
mixture_depth = config.mixture_depth
mixture_width = config.mixture_width
augmix_strength = config.augmix_strength
model_name = config.model_name
features_dl = config.features_dl
resnet_shortcut = config.resnet_shortcut
filters = config.filters
kernel_sizes = config.kernel_sizes
strides = config.strides
pad_value = config.pad_value
lrelu_alpha = config.lrelu_alpha
pooling_conv_filters = config.pooling_conv_filters
perform_pooling = config.perform_pooling
linear_units = config.linear_units
dropout_p = config.dropout_p
use_bias = config.use_bias
num_classes = config.num_classes
num_ohe_classes = config.num_ohe_classes
pretrained_path = config.pretrained_path
weight_init_name = config.weight_init_name
kaiming_a = config.kaiming_a
kaiming_mode = config.kaiming_mode
kaiming_nonlinearity = config.kaiming_nonlinearity
gain = config.gain
optimizer_name = config.optimizer_name
optimizer_name_next = config.optimizer_name_next
loss_function_name = config.loss_function_name
manual_lr = config.manual_lr
base_lr = config.lr
momentum = config.momentum
weight_decay = config.weight_decay
hessian_power = config.hessian_power
use_lookahead = config.use_lookahead
lookahead_k = config.lookahead_k
lookahead_alpha = config.lookahead_alpha
lr_finder_num_iter = config.lr_finder_num_iter
scheduler_name = config.scheduler_name
warmup_batches = config.warmup_batches
T_0 = config.T_0
T_mult = config.T_mult
eta_min = config.eta_min
step_size_up = config.step_size_up
gamma = config.gamma
step_size = config.step_size
grad_max_norm = config.grad_max_norm
label_weights = config.label_weights
label_smoothing = config.label_smoothing
loss_weights = config.loss_weights
nr_of_decimals = config.nr_of_decimals
# max_epochs = config.max_epochs
batch_size = config.batch_size
max_batch_size = config.max_batch_size
patience_lr = config.patience_lr
patience_opt = config.patience_opt
patience = config.patience
# Logistic regression model
patient_id_col = data_preproc_config.patient_id_col
features_lr = data_preproc_config.features
endpoint = data_preproc_config.endpoint
patient_id_length = data_preproc_config.patient_id_length
ijk = 0

sigmoid_act = torch.nn.Sigmoid()
softmax_act = Activations(softmax=True)
to_onehot = AsDiscrete(to_onehot=num_ohe_classes)
mse_metric = MSEMetric()
auc_metric = ROCAUCMetric()

lmn = 0
excl = ''

In [ ]:
model = misc.get_model(model_name=model_name, num_ohe_classes=num_ohe_classes, channels=channels,
                                        depth=depth, height=height, width=width, n_features=n_features,
                                        resnet_shortcut=resnet_shortcut,
                                        num_classes=num_classes, filters=filters, kernel_sizes=kernel_sizes, strides=strides,
                                        pad_value=pad_value,
                                        lrelu_alpha=lrelu_alpha, dropout_p=dropout_p, pooling_conv_filters=pooling_conv_filters,
                                        perform_pooling=perform_pooling, linear_units=linear_units,
                                        clinical_variables_position=config.clinical_variables_position,   ### DANIEL
                                        clinical_variables_linear_units=config.clinical_variables_linear_units,   ### DANIEL
                                        clinical_variables_dropout_p=config.clinical_variables_dropout_p,   ### DANIEL
                                        use_bias=use_bias, pretrained_path= pretrained_path,      ### DANIEL
                                        logger=logger)

In [ ]:
model = DCNN_LReLU

print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

print("Checkpoint state dict:")